In [22]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery google-cloud-vision google-cloud-storage pandas $USER_FLAG -q

In [23]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [2]:
PROJECT_ID = "layodemo"  # @param {type:"string"}

In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [4]:
VISION_AI_REGION = "us-central1"  # @param {type: "string"}

In [5]:
BUCKET_NAME = "trademark-vision"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [6]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

In [ ]:
! gsutil mb -l $VERTEX_AI_REGION -p $PROJECT_ID $BUCKET_URI

In [7]:
! gsutil ls -al $BUCKET_URI

      1882  2023-02-02T05:39:11Z  gs://trademark-vision/apple.png#1675316351654336  metageneration=1
    115645  2023-02-02T05:39:11Z  gs://trademark-vision/audi.png#1675316351924414  metageneration=1
    122392  2023-02-02T05:39:11Z  gs://trademark-vision/bronco.png#1675316351692789  metageneration=1
      4042  2023-02-02T05:39:10Z  gs://trademark-vision/chanel.png#1675316350908201  metageneration=1
    200315  2023-02-02T05:39:10Z  gs://trademark-vision/disney.png#1675316350913597  metageneration=1
      2143  2023-02-02T05:39:11Z  gs://trademark-vision/facebook.png#1675316351333960  metageneration=1
      2726  2023-02-02T05:39:11Z  gs://trademark-vision/google.png#1675316351600313  metageneration=1
     14739  2023-02-02T05:40:59Z  gs://trademark-vision/nike.png#1675316459058750  metageneration=1
     11441  2023-02-02T05:39:11Z  gs://trademark-vision/starbucks.png#1675316351246083  metageneration=1
      1947  2023-02-02T05:39:11Z  gs://trademark-vision/tiktok.png#167531635131686

In [8]:
data = ! gsutil ls -b gs://trademark-vision/*

In [9]:
print(data)

['gs://trademark-vision/apple.png', 'gs://trademark-vision/audi.png', 'gs://trademark-vision/bronco.png', 'gs://trademark-vision/chanel.png', 'gs://trademark-vision/disney.png', 'gs://trademark-vision/facebook.png', 'gs://trademark-vision/google.png', 'gs://trademark-vision/nike.png', 'gs://trademark-vision/starbucks.png', 'gs://trademark-vision/tiktok.png']


In [10]:
# convert to Pandas DataFrame
import pandas as pd
data_pd = pd.DataFrame(data)

In [11]:
#data_pd.to_csv("gs://trademark-vision/data.csv", header=None, index=None)

In [12]:
#Detects logos in the file located in Google Cloud Storage

def detect_logos_uri():
    from google.cloud import bigquery   
    from google.cloud import aiplatform, bigquery, storage, vision
    from google.cloud import vision

    logo_dict = []
    
    for uri in data:
        #print(uri)
        client = vision.ImageAnnotatorClient()
        image = vision.Image()
        image.source.image_uri = uri
        response = client.logo_detection(image=image)
        if response.logo_annotations:
            logos = response.logo_annotations
            #print(logos[0])
            dict = {"location": uri, "description": logos[0].description, "score": logos[0].score}
            logo_dict.append(dict)
        else:
            dict = {"location": uri, "description": "No logo detected", "score": 0}
            logo_dict.append(dict)
    #print(logo_dict)
    df = pd.DataFrame(logo_dict)
    pd.set_option('expand_frame_repr', False)
    print(df)
    
    #Load the data into that dataset bq_demo using Python Client

    print("BigQuery load started")
    
    client = bigquery.Client(project="layodemo")
    dataset_id = 'vision_demo'
    table_id = 'trademark_vision'
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

    job.result()  # Waits for table load to complete.

    print("Load is complete")


In [13]:
detect_logos_uri()

                              location       description     score
0      gs://trademark-vision/apple.png        Apple Inc.  0.987185
1       gs://trademark-vision/audi.png              Audi  0.659119
2     gs://trademark-vision/bronco.png    Denver Broncos  0.902557
3     gs://trademark-vision/chanel.png            Chanel  0.997394
4     gs://trademark-vision/disney.png  No logo detected  0.000000
5   gs://trademark-vision/facebook.png          Facebook  0.821753
6     gs://trademark-vision/google.png            Google  0.986012
7       gs://trademark-vision/nike.png              Nike  0.994266
8  gs://trademark-vision/starbucks.png         Starbucks  0.999133
9     gs://trademark-vision/tiktok.png          Digimind  0.555075
BigQuery load started
Load is complete


In [14]:
data=['gs://trademark-vision/disney.png']

In [15]:
# Detect Web entities with a local image

def detect_web():
    """Detects web annotations given an image."""
    from google.cloud import vision
    from google.cloud import aiplatform, bigquery, storage, vision
    import pandas as pd
    
    logo_dict = []
        
    for uri in data:
        #print(uri)
        client = vision.ImageAnnotatorClient()
        image = vision.Image()
        image.source.image_uri = uri
        response = client.web_detection(image=image)
        annotations = response.web_detection

        if annotations.best_guess_labels:
            for label in annotations.best_guess_labels:
                #print(uri)
                #print('\nBest guess label: {}'.format(label.label))
                #dict = {"location": uri, "Best guess label": label.label}
                #logo_dict.append(dict)
                if annotations.web_entities:
                    for entity in annotations.web_entities:
                        #print('\n\tScore      : {}'.format(entity.score))
                        #print(u'\tDescription: {}'.format(entity.description))
                        dict = {"uri": uri, "best_guess_label": label.label, "score": entity.score, "description": entity.description}
                        logo_dict.append(dict)      
                
        #if annotations.web_entities:
         #   #print('\n{} Web entities found: '.format(len(annotations.web_entities)))
          #  for entity in annotations.web_entities:
           #     print('\n\tScore      : {}'.format(entity.score))
            #    print(u'\tDescription: {}'.format(entity.description))
                
                #print(df)

    #print(logo_dict)
    df = pd.DataFrame(logo_dict)
    pd.set_option('expand_frame_repr', False)
    print(df)
 #Load the data into that dataset bq_demo using Python Client

    print("BigQuery load started")
    
    client = bigquery.Client(project="layodemo")
    dataset_id = 'vision_demo'
    table_id = 'logo_vision'
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

    job.result()  # Waits for table load to complete.

    print("Load is complete")


In [16]:
detect_web()

                                uri          best_guess_label     score        description
0  gs://trademark-vision/disney.png  disney castle silhouette  0.676044     Graphic design
1  gs://trademark-vision/disney.png  disney castle silhouette  0.549907               Logo
2  gs://trademark-vision/disney.png  disney castle silhouette  0.462572               Font
3  gs://trademark-vision/disney.png  disney castle silhouette  0.448600             Design
4  gs://trademark-vision/disney.png  disney castle silhouette  0.436096              Brand
5  gs://trademark-vision/disney.png  disney castle silhouette  0.411357               Text
6  gs://trademark-vision/disney.png  disney castle silhouette  0.372300              White
7  gs://trademark-vision/disney.png  disney castle silhouette  0.360800              Black
8  gs://trademark-vision/disney.png  disney castle silhouette  0.354600  Cinderella Castle
9  gs://trademark-vision/disney.png  disney castle silhouette  0.352800         Silhouette